# Zaptc ("zapticy")
Perturb the state of a node. Propagate potential node state changes downstream. Count the number of every (state0,state1) system-state transition as the network is walked.  Walk for a given number of time-steps. At each node the propagation spreads to all of its downstream nodes.

### Table of Contents <a id=toc>
* [Imports and setup](#init)
* [Create graph](#creategraph)
* [Run Zaptc](#zaptc)
* [Explore results](#results)

## Imports and setup <a id=init>

In [1]:
from random import choice
from collections import Counter
from statistics import mean

import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout, pydot_layout
import numpy as np

import pyphi.data_models as dm
from pyphi.zap_tc import Zaptc

In [2]:
N = 20 # number of Nodes in system
D = 3  # Degree; average number of connections from a node to downstream nodes
S = 2  # number of States per node
T = 10 # number of time steps to follow transitions

p = D/N

print(f'''
{N}\t# number of Nodes
{D}\t# maximum number of Connections from a node
{S}\t# number of States per node
{p}\t# probability of edge creation for each node pairing

{D**10:,}\t# average nodes changed as a result of propagation fan-out fron single node
''')
# {labels} # node labels


20	# number of Nodes
3	# maximum number of Connections from a node
2	# number of States per node
0.15	# probability of edge creation for each node pairing

59,049	# average nodes changed as a result of propagation fan-out fron single node



## Create graph <a id=creategraph>

In [3]:
#g = nx.gnp_random_graph(60, 0.10, directed=True) # num_nodes, prob_edge
G = nx.gnp_random_graph(N, p, directed=True) # num_nodes, prob_edge
edges = list(G.edges())
net = dm.Net(edges=edges)

#Image(filename=png, width=500)
#nx.draw(net.graph, pos=pydot_layout(net.graph), with_labels=True, label='gnp_random_graph({N},{p})')
net.draw()

KeyError: 19

In [ ]:
p = 23  # myst be prime
G = nx.chordal_cycle_graph(p) # an Expander graph
edges = list(G.edges())
net = dm.Net(edges=edges)

net.draw()

In [ ]:
print(nx.info(net.graph))

label0 = sorted(net.graph.nodes())[0]
print()
print(f'label0="{label0}"')
print(f'Downstream nodes of {label0} = {list(net.graph.neighbors(label0))}')

### Change some individual nodes 

#### Add a state to node A (disabled)

#### Set node B to single state (disabled)

## Run Zaptc <a id=zaptc>

In [ ]:
ztc = Zaptc(net=net)
ztc

In [ ]:
len(ztc.net)

In [ ]:
list(ztc.net.successors(label0))

In [ ]:
list(ztc.net.get_node('A').states)

In [ ]:
start_state = '0'*len(ztc.net)
print(f'start_state = "{start_state}", start_label="{label0}"\n')
%time ztc.zap_tc(label0, start_state, T)

## Results <a id=results>

In [ ]:
print(nx.info(net.graph))

In [ ]:
nstates = len(set([s1 for s1,s2 in ztc.transition_counter.keys()]).intersection([s2 for s1,s2 in ztc.transition_counter.keys()]))
print(f'Number of unique transitions = {len(ztc.transition_counter)}')
print(f'total transitions = {sum(ztc.transition_counter.values())}')
print(f'num system-states = {nstates}')
# state_list

In [ ]:
print(f'{len(ztc.transition_counter):,}\tUnique transitions observed')

In [ ]:
%time ztc.zapall(T)

In [ ]:
instates = set(s1 for s1,s2 in ztc.transition_counter.keys())
outstates = set(s2 for s1,s2 in ztc.transition_counter.keys())
#nstates = len(instates.union(outstates))
maxnstates = S**len(ztc.net)
print(f'''
{len(instates):,}  \t Number of IN  states encountered
{len(outstates):,} \t Number of OUT  states encountered


{len(instates-outstates):,}  \t Number of IN-OUT states encountered
{len(outstates-instates):,}  \t Number of OUT-IN states encountered
{len(outstates&instates):,}  \t Number of COMMON (in/out) states encountered
=======
{len(outstates|instates):,}  \t Number of COMBINED (in/out) states encountered

{maxnstates:,}\t Number of possible states

{len(instates)*len(outstates):,} \t Values needed for rectangular TPM 
{len(ztc.transition_counter):,}\t\t Unique transitions observed (compare to rectangular TPM size)
{sum(ztc.transition_counter.values()):,}\t TOTAL transitions observed

{maxnstates**2:,}\t Number of POSSIBLE transitions
''')

In [ ]:
ztc.transition_counter

In [ ]:
ztc.tpm_sbn

In [ ]:
ztc.net.nodes

In [ ]:
list(G.edges.items())[0]

In [ ]:
len(net.nodes)

In [ ]:
ztc.tpm_sbn()